In [22]:
import pandas as pd
import numpy as np
import os
import re

In [24]:
cpt_file = r"C:\Users\hp\OneDrive - University of Hertfordshire\CPT_II_ConnersContinuousPerformanceTest.csv"
cpt_df = pd.read_csv(cpt_file, sep=";")
cpt_df.head()

,ID,Assessment Status,Assessment Duration,Type,LastTrial,Trial1,Response1,Trial2,Response2,Trial3,...,Raw Score HitRTIsi,Percent HitRTIsi,General TScore HitSEIsi,Adhd TScore HitSEIsi,Neuro TScore HitSEIsi,Raw Score HitSEIsi,Percent HitSEIsi,Adhd Confidence Index,Neuro Confidence Index,Old Overall Index
0,2,3,839999,6,360,11,-1,13,469,7,...,0.049592,0,48.021103,47.772537,43.224297,-0.021096,0,61.145496,58.749630,10.070000
1,3,3,839999,6,360,12,-1,0,371,2,...,0.064090,0,42.234097,38.263283,41.574574,-0.077351,0,86.871597,61.088688,4.440000
2,4,3,839999,6,360,14,-1,10,-1,3,...,0.077233,0,45.680782,42.289310,43.868061,-0.046142,0,99.900002,86.131813,18.340000
3,5,3,839999,6,360,12,459,7,289,12,...,0.085540,0,85.698700,70.469986,80.228539,0.410634,0,99.900002,99.900002,8.950001
4,7,3,839998,6,360,11,841,7,-1,5,...,0.024632,0,42.058998,43.349667,39.992119,-0.071200,0,32.291084,24.225414,0.000000


In [26]:
len(cpt_df.columns)

786

In [28]:
#Lets do some basic cleaning and rename few things (This is optional)
cpt_df.rename(columns={'ID':'Participant_ID'}, inplace=True)

rename_map = {
    'General TScore Omission' : 'General_TScore_Omission',
    'Adhd TScore Omission' : 'Adhd_TScore_Omission',
    'Neuro TScore Omission' : 'Neuro_TScore_Omission',
    'Adhd Confidence Index' : 'Adhd_Confidence_Index',
    'Neuro Confidence Index' : 'Neuro_Confidence_Index',
    'Old Overall Index' : 'Old_Overall_Index'
}
cpt_df.rename(columns=rename_map, inplace=True)

In [30]:
#This is to Identify columns for trials vs. summary
#Since the dataset has many columns of the form
#    For example the data has Trial1,Response1,Trial2,Response2,..., Trial360, Response360,..
#    And also columns like 'General_TScore_Omission','Adhd_Confidence_Index', Etc..
# So lets seperate them into ID + test-level summary columns


In [32]:
def is_trial_or_response(col):
    #Example of matches: "Trial1", "Trial2", "Trial360", "Response10", etc.
    # We'll rely on the fact that it starts with "Trial" or "Response" followed by a number.
    if col.startswith("Trial") or col.startswith("Response"):
        # The rest of the string after "Trial" or "Response" is an integer
        suffix = col.replace("Trial", "").replace("Response", "")
        return suffix.isdigit()
    return False
    

In [34]:
# To Identify the columns that are trial/response columns
trial_cols = [c for c in cpt_df.columns if is_trial_or_response(c)]
# To Identify the columns that are not trial/response
summary_cols = [c for c in cpt_df.columns if c not in trial_cols]

#We'll keep the summary columns in one DataFrame
cpt_summary_df = cpt_df[summary_cols].copy()

#And we'll keep trial columns in another for pivoting
cpt_trials_df = cpt_df[["Participant_ID"] + trial_cols].copy()


In [36]:
# 4) Convert the wide trial/response columns into long fromat
# We want each row to be in the final DataFrame to look like:
#   participant_ID | TrialNumber | TrialValue | ResponseValue
#
# E.g., for columns "Trial1" and "Response1", that becomes (1, trial_val, response_val).
# We can do this in a few ways; one approach is to reshape using pandas wide_to_long or melt.

# Approach A: Using 'melt' + a bit of manipulation
# We'll create two "melted" frames: one for Trials, one for Responses.
trial_columns = [col for col in cpt_trials_df.columns if col.startswith("Trial")]
response_columns = [col for col in cpt_trials_df.columns if col.startswith("Response")]

# Melt trials: "Trial1" -> a row with variable="Trial1", value=<trial value>
trial_melted = cpt_trials_df.melt(
    id_vars="Participant_ID",
    value_vars=trial_columns,
    var_name="TrialVar",
    value_name="TrialValue"
)

# Melt responses: "Response1" -> a row with variable="Response1", value=<response value>
response_melted = cpt_trials_df.melt(
    id_vars="Participant_ID",
    value_vars=response_columns,
    var_name="ResponseVar",
    value_name="ResponseValue"
)

In [38]:
# Each melted DataFrame has "Participant_ID", "TrialVar"/"ResponseVar", and "TrialValue"/"ResponseValue".
# We need to merge them such that "Trial1" lines up with "Response1".
# One trick: notice the digit portion of "Trial1" should match "Response1".
# We'll parse out that digit, so we can do a join on [Participant_ID, trial_number].

# Parse the integer from "Trial1" or "Response1"
trial_melted["TrialNumber"] = trial_melted["TrialVar"].str.extract(r'(\d+)').astype(int)
response_melted["TrialNumber"] = response_melted["ResponseVar"].str.extract(r'(\d+)').astype(int)

# Now we can merge them on Participant_ID + TrialNumber
cpt_long = pd.merge(
    trial_melted[["Participant_ID", "TrialNumber", "TrialValue"]],
    response_melted[["Participant_ID", "TrialNumber", "ResponseValue"]],
    on=["Participant_ID", "TrialNumber"],
    how="outer"
)

In [40]:
# now I think it will be fine to merge the cpt_long with the summary colums
# The summary columns are such as TScore, Confidence Indices, etc. are participant-level.
# 

In [42]:
cpt_long_merged = pd.merge(
    cpt_long,
    cpt_summary_df,
    on="Participant_ID",
    how="left"
)

In [44]:
# 6) Convert data types, handle missing values, etc.
# ------------------------------------------------------------------------------
# For instance, ensure trial and response columns are numeric if appropriate.
cpt_long_merged["TrialValue"] = pd.to_numeric(cpt_long_merged["TrialValue"], errors="coerce")
cpt_long_merged["ResponseValue"] = pd.to_numeric(cpt_long_merged["ResponseValue"], errors="coerce")

# Example: fill any missing numeric data with 0 or NaN as needed
# cpt_long_merged["TrialValue"].fillna(0, inplace=True)

# 7) Final check
# ------------------------------------------------------------------------------
print("CPT long format shape:", cpt_long_merged.shape)
print(cpt_long_merged.head(10))

print("\nSummary-level columns included:", [col for col in cpt_summary_df.columns])

CPT long format shape: (35640, 69)
   Participant_ID  TrialNumber  TrialValue  ResponseValue  Assessment Status  \
0               2            1          11             -1                  3   
1               2            2          13            469                  3   
2               2            3           7            343                  3   
3               2            4          15            275                  3   
4               2            5           0            295                  3   
5               2            6          12            266                  3   
6               2            7           2            246                  3   
7               2            8           1            279                  3   
8               2            9           9            337                  3   
9               2           10           5            328                  3   

   Assessment Duration  Type  LastTrial  General TScore Omissions  \
0              